In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Starry Net Forecasting Pipeline

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/forecasting/starry_net_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fforecasting%2Fstarry_net_pipeline.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/forecasting/starry_net_pipeline.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/forecasting/starry_net_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

### Overview

In this tutorial, you learn how to create a Starry Net forecasting model using [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) downloaded from [Google Cloud Pipeline Components](https://cloud.google.com/vertex-ai/docs/pipelines/components-introduction) (GCPC). Starry Net is a state-of-the-art forecaster developed and used internally by Google. Starry Net is a glass-box neural network inspired by statistical time series models, capable of cleaning step changes and spikes, modeling seasonality and events, forecasting trend, and providing both point and prediction interval forecasts in a single, lightweight model. Starry Net stands out among neural network based forecasting models by providing the explainability, interpretability and tunability of traditional statistical forecasters. For example, it features time series feature decomposition and damped local linear exponential smoothing model as the trend structure.

This tutorial uses the following Google Cloud ML services:

- AutoML training
- Vertex AI Pipelines

The steps performed are:

- Create a training pipeline with Starry Net.
- Perform the batch prediction using the trained model in the above step.
- Deploy the trained model to an endpoint to perform online predictions and generate decomposition plots.

### Dataset

This tutorial uses the [Liquor dataset](https://www.kaggle.com/datasets/residentmario/iowa-liquor-sales), which forecasts the alcoholic beverage sales in the Midwest.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* BigQuery
* Dataflow

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and [BigQuery](https://cloud.google.com/bigquery), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Install additional packages

Install the Google Cloud Pipeline Components (GCPC) SDK not earlier than `2.16.1`.


In [ ]:
!pip3 install --upgrade \
    google-cloud-pipeline-components==2.16.1 \
    grpcio==1.62.1 \
    grpcio-status==1.48.2 \
    googleapis-common-protos==1.63.0 \
    grpc-google-iam-v1==0.13.0 \
    google-crc32c==1.5.0 \
    kfp==2.7.0 \
    protobuf==4.25.3

## Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager).

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,dataflow.googleapis.com,compute_component,storage-component.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

## Notes about service account and permission

For full details of the permission setup, refer to https://cloud.google.com/vertex-ai/docs/tabular-data/tabular-workflows/service-accounts

**By default no configuration is required**, if you run into any permission related issue, please make sure the service accounts above have the required roles:

|Service account email|Description|Roles|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|Compute Engine default service account|Dataflow Developer, Dataflow Worker, Storage Admin, BigQuery Data Editor, Vertex AI User, Service Account User|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI Platform Service Agent|Vertex AI Service Agent|


1. Goto https://console.cloud.google.com/iam-admin/iam.
2. Check the "Include Google-provided role grants" checkbox.
3. Find the above emails.
4. Grant the corresponding roles.

### Using data source from a different project
- For the BQ data source, grant both service accounts the "BigQuery Data Viewer" role.
- For the CSV data source, grant both service accounts the "Storage Object Viewer" role.


### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### Location

You can also change the `LOCATION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
LOCATION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing since you're already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

## Import libraries and define constants

In [ ]:
# Import required modules

import math
import os
import time
from typing import Any, Dict, List, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.cloud import aiplatform, bigquery
from google_cloud_pipeline_components.preview.starry_net import starry_net
from kfp import compiler

## Initialize Vertex AI SDK for Python

Initialize the Vertex SDK for Python for your project.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION)

## Create the dataset

In [ ]:
now = str(int(time.time()))
bigquery_dataset = f"starry_net_test_{now}"
bigquery_table = "iowa_weekly_by_store"

client = bigquery.Client()
dataset = bigquery.Dataset(f"{PROJECT_ID}.{bigquery_dataset}")
dataset.location = "US"  # Location must be in the multi-region US, since the source dataset is in multi-region US.
client.create_dataset(dataset, timeout=30)

query = f"""
CREATE OR REPLACE TABLE {PROJECT_ID}.{bigquery_dataset}.{bigquery_table} as
SELECT 
DATE_TRUNC(date, WEEK) as week,
SUM(sale_dollars) as sales,
CONCAT(store_name, "|", store_number) as item,
store_name, store_number, city, county,
FROM `bigquery-public-data.iowa_liquor_sales.sales`
GROUP BY week, item, store_name, store_number, city, county
"""

_ = client.query(query).result()

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets, TF model checkpoint, TensorBoard file, etc.

In [ ]:
BUCKET_URI = f"gs://{PROJECT_ID}-starry-net-{now}"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

We create the bucket in multi-region US, since the bucket and the Big Query dataset must exist in the same region.

In [ ]:
! gsutil mb -l US -p {PROJECT_ID} {BUCKET_URI}

## Compile the pipeline

In [ ]:
template_path = os.path.join(os.getcwd(), "starry-net.yaml")
compiler.Compiler().compile(starry_net, template_path)

## Create TensorBoard Instance for saving decomposition plots

NOTE: This can be done in the console. See [documentation](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-setup#google-cloud-console) for details.

In [ ]:
def create_tensorboard(
    project: str,
    location: str,
    display_name: Optional[str] = None,
):
    aiplatform.init(project=project, location=location)

    tensorboard = aiplatform.Tensorboard.create(
        display_name=display_name,
        project=project,
        location=location,
    )

    aiplatform.init(
        project=project, location=location, experiment_tensorboard=tensorboard
    )

    return tensorboard


tensorboard = create_tensorboard(PROJECT_ID, LOCATION, "starry-net-tensorboard")

## Set the pipeline parameters

In terms of tuning your model, the most important parameters are

```
dataprep_backcast_length: The length of the context window to feed into the
  model. Generally, this can be somewhere bween 2-4x forecast horizon.
  The shorter, the more reactive the model will be.

dataprep_forecast_length: The length of the forecast horizon used in the
  loss function during training and during evaluation, so that the model is
  optimized to produce forecasts from 0 to H.

trainer_model_blocks: The list of model blocks to use in the order they will
  appear in the model. Possible values are `cleaning`, `change_point`,
  `trend`, `hour_of_week`, `day_of_week`, `day_of_year`, `week_of_year`,
  `month_of_year`, `residual`.
  Model blocks should always include `cleaning`, `change_point`,
  `trend`, and `residual`, in that order. If your data has seasonal patterns,
  add any of the relevant seasonal blocks between the `trend` and `residual`
  blocks.
  
trainer_cleaning_activation_regularizer_coeff: The L1 regularization
  coefficient for the anomaly detection activation in the cleaning block.
  The larger the value, the less aggressive the cleaning, so fewer and only
  the most extreme anomalies are detected. A rule of thumb is that this
  value should be about the same scale of your series.

trainer_change_point_activation_regularizer_coeff: The L1 regularization
  coefficient for the change point detection activation in the change point
  block. The larger the value, the less aggressive the cleaning, so fewer
  and only the most extreme change points are detected. A rule of thumb is
  that this value should be a ratio of the
  trainer_change_point_output_regularizer_coeff to determine the sparsity
  of the changes. If you want the model to detect many small step changes
  this number should be smaller than the
  trainer_change_point_output_regularizer_coeff. To detect fewer large step
  changes, this number should be about equal to or larger than the
  trainer_change_point_output_regularizer_coeff.

trainer_change_point_output_regularizer_coeff: The L2 regularization
  penalty applied to the mean lag-one difference of the cleaned output of
  the change point block. Intutively,
  trainer_change_point_activation_regularizer_coeff determines how many
  steps to detect in the series, while this parameter determines how
  aggressively to clean the detected steps. The higher this value, the more
  aggressive the cleaning. A rule of thumb is that this value should be
  about the same scale of your series.
 ```

In [ ]:
root_dir = os.path.join(BUCKET_URI, f"automl_forecasting_pipeline/run-{now}")
parameters = {
    "tensorboard_instance_id": tensorboard.resource_name.split("/")[-1],
    "dataprep_backcast_length": 80,
    "dataprep_forecast_length": 12,
    "dataprep_train_end_date": "2023-11-01",
    "dataprep_n_val_windows": 1,
    "dataprep_n_test_windows": 1,
    "dataprep_test_set_stride": 4,
    "dataprep_test_set_bigquery_dataset": f"bq://{PROJECT_ID}.{bigquery_dataset}",
    "dataflow_machine_type": "n1-standard-16",
    "dataflow_max_replica_count": 50,
    "dataflow_starting_replica_count": 25,
    "dataflow_disk_size_gb": 50,
    "dataprep_csv_data_path": "",
    "dataprep_csv_static_covariates_path": "",
    "dataprep_bigquery_data_path": f"bq://{PROJECT_ID}.{bigquery_dataset}.{bigquery_table}",
    "dataprep_ts_identifier_columns": ["item"],
    "dataprep_time_column": "week",
    "dataprep_target_column": "sales",
    "dataprep_static_covariate_columns": ["county", "city"],
    "dataprep_previous_run_dir": "",
    "dataprep_nan_threshold": 0.95,
    "dataprep_zero_threshold": 0.95,
    "trainer_machine_type": "n1-standard-8",
    "trainer_accelerator_type": "NVIDIA_TESLA_V100",
    "trainer_num_epochs": 1,
    "trainer_cleaning_activation_regularizer_coeff": 1e3,
    "trainer_change_point_activation_regularizer_coeff": 1e3,
    "trainer_change_point_output_regularizer_coeff": 1e3,
    "trainer_trend_alpha_upper_bound": 0.5,
    "trainer_trend_beta_upper_bound": 0.2,
    "trainer_trend_phi_lower_bound": 0.99,
    "trainer_trend_b_fixed_val": -1,
    "trainer_trend_b0_fixed_val": -1,
    "trainer_trend_phi_fixed_val": -1,
    "trainer_quantiles": [],
    "trainer_model_blocks": [
        "cleaning",
        "change_point",
        "trend",
        "week_of_year",
        "month_of_year",
        "residual",
    ],
    "tensorboard_n_decomposition_plots": 5,
    "encryption_spec_key_name": "",
    "location": LOCATION,
    "project": PROJECT_ID,
}

## Launch the pipeline

NOTE: This can be done in the console straight for the [Vertex AI Template Gallery](https://cloud.google.com/vertex-ai/docs/pipelines/use-template-gallery#console) by seraching for the Starry Net pipeline.

In [ ]:
job_id = f"starry-net-{now}"
job = aiplatform.PipelineJob(
    display_name=job_id,
    location=LOCATION,
    template_path=template_path,
    job_id=job_id,
    pipeline_root=root_dir,
    parameter_values=parameters,
    enable_caching=False,
)
job.run()

## Create the Model Endpoint and Deploy Model

NOTE: This can all be done in the console. See [documentation](https://cloud.google.com/vertex-ai/docs/general/deployment#google-cloud-console) for details.

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name="starry_net_test",
    project=PROJECT_ID,
    location=LOCATION,
)

print(endpoint.display_name)
print(endpoint.resource_name)

In [ ]:
model_name = None
for task in job.task_details:
    if "model-upload" in task.task_name:
        for k, v in task.outputs.items():
            if k == "model":
                model_name = v.artifacts[0].uri.split("/v1/")[1]
if model_name is None:
    raise ValueError(
        "Failed to find model. Try looking up the name in the Model Registry console."
    )

model = aiplatform.Model(model_name=model_name)

model.deploy(
    endpoint=endpoint,
    deployed_model_display_name="My Starry test model",
    traffic_percentage=100,
    machine_type="n1-standard-8",
    min_replica_count=1,
    max_replica_count=1,
)

model.wait()

print(model.display_name)
print(model.resource_name)

## Helper Functions for Plotting Forecasts

In [ ]:
_COLOR_STYLE = {
    "history": "dodgerblue",
    "input remainder": "dodgerblue",
    "fitted": "red",
    "forecasts": "limegreen",
    "forecasts_P50": "limegreen",
    "forecasts_P95": "darkviolet",
}


def _plot_forecast(
    backcast_len: int,
    forecast_len: int,
    timestamps: Optional[List[pd.Timestamp]],
    history: List[float],
    fitted: List[float],
    forecasts: List[float],
    color_style: Dict[str, str],
    ax: Optional[plt.Axes],
) -> None:
    """Plots the overall forecasts from a Starry-Net Vertex Endpoint."""
    forecasts = [np.nan] * backcast_len + forecasts
    fitted = fitted + [np.nan] * forecast_len
    history = history + [np.nan] * forecast_len
    data = {"forecasts_P50": forecasts, "fitted": fitted, "history": history}
    df = pd.DataFrame(data, index=timestamps)
    df.plot(
        kind="line",
        ax=ax,
        legend=True,
        title="Overall forecasts",
        style=color_style,
    )


def _plot_cleaning_block(
    block_name: str,
    timestamps: Optional[List[pd.Timestamp]],
    input_remainder: List[float],
    cleaned_series: List[float],
    color_style: Dict[str, str],
    legend: bool,
    ax: Optional[plt.Axes],
) -> None:
    """Plots a cleaning block's output from a Starry-Net Vertex Endpoint."""
    data = {
        "input remainder": input_remainder,
        "fitted": cleaned_series,
    }
    df = pd.DataFrame(data, index=timestamps)
    df.plot(kind="line", ax=ax, legend=legend, title=block_name, style=color_style)


def _plot_intermediate_block(
    block_name: str,
    backcast_len: int,
    forecast_len: int,
    timestamps: Optional[List[pd.Timestamp]],
    input_remainder: List[float],
    fitted: List[float],
    forecasts: List[float],
    color_style: Dict[str, str],
    legend: bool,
    ax: Optional[plt.Axes],
) -> None:
    """Plots an intermediate block's output from a Starry-Net Vertex Endpoint."""
    forecasts = [np.nan] * backcast_len + forecasts
    fitted = fitted + [np.nan] * forecast_len
    input_remainder = input_remainder + [np.nan] * forecast_len
    data = {
        "input remainder": input_remainder,
        "fitted": fitted,
        "forecasts": forecasts,
    }
    df = pd.DataFrame(data, index=timestamps)
    df.plot(kind="line", ax=ax, legend=legend, title=block_name, style=color_style)


def plot_forecasts_from_endpoint(
    instance: Dict[str, Any],
    predictions: Dict[str, Any],
    height: float = 2.5,
    width: float = 5,
    color_style: Optional[Dict[str, str]] = None,
) -> None:
    """Plots forecasts from a Starry-Net Vertex AI endpoint.

    Args:
      instance: The input instance sent to the end point containing the input time
        series and maybe timestamps.
      predictions: The dictionary of predictions.
      height: height of the subplot in the visualization.
      width: width of the subplot in the visualization.
      color_style: The map of colors to use for each plot.
    """
    # set up subplot grid
    i = 2
    j = 0
    decomposition = predictions.get("decomposition", {})
    blocks = sorted(decomposition.keys())
    n_blocks = len(blocks)
    row = math.ceil(n_blocks / 2) + 2
    fig_size = (2 * width, row * height)
    fig, axes = plt.subplots(row, 2, figsize=fig_size, sharey=True)
    fig.tight_layout(pad=3.0)
    # plot overall forecast
    ax_overall = plt.subplot2grid((row, 2), (0, 0), colspan=2, rowspan=2)
    axes[0, 0].tick_params(labelbottom=False, labelleft=False)
    axes[0, 1].tick_params(labelbottom=False, labelleft=False)
    axes[1, 0].tick_params(labelbottom=False, labelleft=False)
    axes[1, 1].tick_params(labelbottom=False, labelleft=False)

    timestamps = instance.get("timestamps")
    backcast_len = len(predictions["fitted"])
    forecast_len = len(predictions["value"])
    total_len = backcast_len + forecast_len
    x = instance["x"]
    if len(x) != backcast_len:
        raise ValueError(f'instance["x"] should be of len {backcast_len}')
    if timestamps is not None:
        timestamps = [pd.to_datetime(ts) for ts in timestamps]
        if len(timestamps) != total_len:
            raise ValueError(f'instance["timestamps"] should be of len {total_len}')
    else:
        timestamps = (
            range(backcast_len + forecast_len) if timestamps is None else timestamps
        )

    _plot_forecast(
        backcast_len,
        forecast_len,
        timestamps,
        x,
        predictions["fitted"],
        predictions["value"],
        color_style or _COLOR_STYLE,
        ax_overall,
    )
    for idx, name in enumerate(blocks):
        axs = axes[i, j]
        if "cleaning" in name or "change_point" in name:
            _plot_cleaning_block(
                name.split(":")[1],
                timestamps[:backcast_len],
                decomposition[name]["input remainder"],
                decomposition[name]["cleaned_series"],
                color_style or _COLOR_STYLE,
                idx % 2 == 1 or idx == len(blocks) - 1,
                axs,
            )
        else:
            _plot_intermediate_block(
                name.split(":")[1],
                backcast_len,
                forecast_len,
                timestamps,
                decomposition[name]["input remainder"],
                decomposition[name]["fitted"],
                decomposition[name]["forecast"],
                color_style or _COLOR_STYLE,
                idx % 2 == 1 or idx == len(blocks) - 1,
                axs,
            )
        if j < 1:
            j += 1
        else:
            i += 1
            j = 0
    if len(blocks) % 2 != 0:
        # delete empty axes
        fig.delaxes(axes.flatten()[row * 2 - 1])

## Query test set to get an input time series

In [ ]:
client = bigquery.Client()
table_id = None
for task in job.task_details:
    if task.task_name == "set-test-set":
        for k, v in task.outputs.items():
            if k == "artifact":
                table_id = v.artifacts[0].uri[5:]
                print(f"test set table id: {table_id}")
if table_id is None:
    raise ValueError(
        "Failed to find test set BQ TABLE. Try looking up the name in the pipeline DAG in the Vertex Console."
    )

query = f"SELECT * FROM `{table_id}` LIMIT 10"
for row in client.query(query).result():
    break

## Call Endpoint to Generate Forecasts

In [ ]:
instance = {
    "x": row.x,
    "timestamps": row.timestamps,
    "city": row.city,
    "county": row.county,
}
out = endpoint.predict(instances=[instance])
out.predictions[0]["value"]

## Call Endpoint to Generate Decompositions and Visualize Decomposition Plots

In [ ]:
out = endpoint.predict(instances=[instance], parameters={"include_decomposition": True})
plot_forecasts_from_endpoint(instance, out.predictions[0])

## Clean Up Resources

You can delete all the resources you created with the following.

Be careful not to accidentally delete resources you might use outside of this demo.

If you did not create a brand new bucket and BQ dataset, then do not set `DELETE_BQ_DATASET` and `DELETE_BUCKET` to `True`.

In [ ]:
DELETE_VERTEX_RESOURCES = True
DELETE_BQ_DATASET = True
DELETE_BUCKET = True
if DELETE_VERTEX_RESOURCES:
    # Undeploy model from endpoint
    endpoint.undeploy_all()

    # Delete model
    model.delete()

    # Delete endpoint
    endpoint.delete()

    # Delete the job
    job.delete()

    # Delete the tensorboard instance
    tensorboard.delete()

if DELETE_BQ_DATASET:
    # Delete the BQ dataset
    client.delete_dataset(dataset, delete_contents=True)

if DELETE_BUCKET:
    # Delete the bucket
    ! gsutil -m rm -r $BUCKET_URI